In [1]:
import os
import sys

import numpy as np

import pandas as pd
import glob

In [2]:
BASE_MODEL_PATH="trained_model"
%mkdir -p $BASE_MODEL_PATH

In [3]:
from models.modelutils import ModelCompiler

Using TensorFlow backend.


In [4]:
compiler = ModelCompiler(BASE_MODEL_PATH)

In [5]:
from models.processor import create_generators

TRAIN_DATAGEN, VALID_DATAGEN = create_generators()

In [6]:
from models.modelutils import dir2filedict, split_fdict

In [7]:
fdict = dir2filedict("data")

In [21]:
categories = sorted(fdict.keys())

In [9]:
trdict, testdict = split_fdict(fdict, test_size=0.2, random_state = 123)

In [10]:
trdict, valdict = split_fdict(trdict, test_size=0.2, random_state = 456)

In [11]:
valdict['clouds'][0:5]

['data/clouds/0678.jpeg',
 'data/clouds/0701.jpeg',
 'data/clouds/0431.jpeg',
 'data/clouds/0033.jpeg',
 'data/clouds/0290.jpeg']

In [12]:
from models.one_vs_all import OneVsAllModelTrainer
from models.modelutils import split_files

In [13]:
trainer = OneVsAllModelTrainer(TRAIN_DATAGEN, VALID_DATAGEN)

In [14]:
def train_one_category(cat, epoch=5):
    model_path = "{}/model_{}".format(BASE_MODEL_PATH, cat)
    model = compiler.generate_compiled_model(model_path)
    
    trainer.set_model(model)
    trainer.set_savepath(model_path)
    
    true_train, false_train = split_files(cat, trdict)
    true_valid, false_valid = split_files(cat, valdict)
    
    trainer.set_dataset_files(true_train, false_train, true_valid, false_valid)
    trainer.train_model(eachepochs=epoch)

In [18]:
list(range(0, 6))

[3, 4, 5, 6]

In [ ]:
for i in range(0, 6):
    train_one_category(categories[i])

In [ ]:
for i in range(6, 12):
    train_one_category(categories[i])

In [20]:
for i in range(12, len(categories)):
    train_one_category(categories[i])

16

### Try and error

In [64]:
cat = "bay"

In [65]:
model_path = "{}/model_{}".format(BASE_MODEL_PATH, cat)

In [66]:
model = compiler.generate_compiled_model(model_path)

In [67]:
trainer.set_model(model)
trainer.set_savepath(model_path)

In [68]:
true_train, false_train = split_files(cat, trdict)
true_valid, false_valid = split_files(cat, valdict)

In [69]:
trainer.set_dataset_files(true_train, false_train, true_valid, false_valid)

In [70]:
trainer.train_model()

Epoch 1/5
51/98 [==============>...............] - ETA: 1:12 - loss: 0.3607 - acc: 0.8493

/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 80000 bytes but only got 0. Skipping tag 64640
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3368026112 bytes but only got 0. Skipping tag 7
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 134479872 bytes but only got 0. Skipping tag 7
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4294901760 bytes but only got 0. Skipping tag 4
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expec

54/98 [===============>..............] - ETA: 1:06 - loss: 0.3620 - acc: 0.8484

/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1572864 bytes but only got 0. Skipping tag 42
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19600506880 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2684485632 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2751528960 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expe

81/98 [=======================>......] - ETA: 25s - loss: 0.3932 - acc: 0.8333

/usr/local/lib/python3.5/dist-packages/PIL/Image.py:2496: DecompressionBombWarning: Image size (103926732 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


99/98 [==============================] - 301s 3s/step - loss: 0.3884 - acc: 0.8356 - val_loss: 0.2745 - val_acc: 0.8988
Epoch 2/5
99/98 [==============================] - 216s 2s/step - loss: 0.2750 - acc: 0.8830 - val_loss: 0.3250 - val_acc: 0.8828
Epoch 3/5
99/98 [==============================] - 220s 2s/step - loss: 0.2554 - acc: 0.8964 - val_loss: 0.3687 - val_acc: 0.8685
Epoch 4/5
99/98 [==============================] - 216s 2s/step - loss: 0.2180 - acc: 0.9058 - val_loss: 0.3158 - val_acc: 0.8786
Epoch 5/5
99/98 [==============================] - 214s 2s/step - loss: 0.2154 - acc: 0.9052 - val_loss: 0.3626 - val_acc: 0.8769
